In [46]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook', font_scale=1.3)

import itertools as it
from collections import Counter, defaultdict, deque

from oldmouselab import OldMouselabEnv
from policies import FixedPlanPolicy, FunctionPolicy
from agents import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# 2 by 2 env with 8-valued discrete rewards
env = OldMouselabEnv(2, 2, quantization=8, cost=0.1, ground_truth=False)
agent = Agent()
agent.register(env)

In [126]:
# Exactly compute V(init)
from exact import solve
def hash_state(s):
    return sum(hash(tuple(str(x) for x in s[i: i+env.outcomes]))
                 for i in range(0, len(env.init), env.outcomes))
Q, V, pi, info = solve(env, hash_state=hash_state)
print(V(env.init))
print(info)

12195281018674982810

In [130]:
from scipy.special import comb

def n_states(gambles, outcomes, quant):
    return comb((quant+1)**outcomes, gambles, repetition=True)

n_states(2, 3, 8)

266085.0

In [128]:
# Empirically estimate V(init) for the exact optimal policy
agent.register(FunctionPolicy(pi))
df = pd.DataFrame(agent.run_many(5000))
df['return'].mean()

HBox(children=(IntProgress(value=0, max=5000), HTML(value='')))

1.1269142324921697

In [45]:
# Empirically estimate V(init) for full observation policy
# This policy is optimal when cost=0
agent.register(FixedPlanPolicy(range(5)))
df = pd.DataFrame(agent.run_many(5000))
df['return'].mean()

HBox(children=(IntProgress(value=0, max=5000), HTML(value='')))

1.0587398231212921

In [49]:
envs = [OldMouselabEnv(2, 2, quantization=8, cost=0.1)
        for _ in range(100)]

from evaluation import bo_policy
policy, result = bo_policy(n_calls=30, return_result=True, n_jobs=20)

TypeError: bo_policy() missing 1 required positional argument: 'envs'